In [1]:
!pip install pdfplumber

In [2]:
import pandas as pd

In [3]:
import pdfplumber
import re

# Открытие PDF файла
with pdfplumber.open(r"C:\Users\lanmo\Downloads\Braking_EN.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
# Разделение текста по пунктам (предположим, что пункты начинаются с цифры и точки, например "1.")
sections = re.split(r'(\n5(?:\.\d+)+\.)', text)


In [5]:
sections = sections[1:]

In [6]:
df = {'punkt':[sections[i].replace('\n','') for i in range(len(sections)) if i%2 == 0],
     'text':[sections[i] for i in range(len(sections)) if i%2 == 1]}

In [7]:
dt = pd.DataFrame(df)

In [8]:
dt.head(5)

,punkt,text
0,5.1.,General
1,5.1.1.,Braking equipment
2,5.1.1.1.,"The braking equipment shall be so designed, c..."
3,5.1.1.2.,"In particular, the braking equipment shall be..."
4,5.1.1.3.,Brake linings shall not contain asbestos.


In [9]:
dt['section1'] = dt['punkt'].str.extract(r'(5\.\d)')
dt['section2'] = dt['punkt'].str.extract(r'(5\.\d+\.\d+)')
dt['section3'] = dt['punkt'].str.extract(r'(5\.\d+\.\d+\.\d+)')
dt['section4'] = dt['punkt'].str.extract(r'(5\.\d+\.\d+\.\d+\.\d+)')
dt.fillna('-1',inplace = True)

In [10]:
def get_main_text(row, df):
    main_section = row['punkt'].split('.')[0] + '.' + row['punkt'].split('.')[1]  # Берем первую часть '5.1' или '5.2'
    match = df.loc[df['section1'] == main_section, 'text']
    return match.values[0] if not match.empty else None

def get_main_text1(row, df):
    # Формируем main_section для уровня '5.1.1'
    parts = row['punkt'].split('.')
    if len(parts) >= 3:
        main_section = f"{parts[0]}.{parts[1]}.{parts[2]}"  # Берем первые три части
    else:
        return None  # Возвращаем None, если меньше трех уровней

    # Находим совпадение по section1 и возвращаем текст
    match = df.loc[df['section2'] == main_section, 'text']
    return match.values[0] if not match.empty else None

def get_main_text2(row, df):
    # Формируем main_section для уровня '5.1.1.1'
    parts = row['punkt'].split('.')
    if len(parts) >= 4:
        main_section = f"{parts[0]}.{parts[1]}.{parts[2]}.{parts[3]}"  # Берем первые четыре части
    else:
        return None  # Возвращаем None, если меньше четырех уровней



    # Находим совпадение по section2 и возвращаем текст
    match = df.loc[df['section3'] == main_section, 'text']

    return match.values[0] if not match.empty else None

def get_main_text_5(row, df):
    # Формируем main_section для уровня '5.1.1.1.1'
    parts = row['punkt'].split('.')
    if len(parts) >= 5:
        main_section = f"{parts[0]}.{parts[1]}.{parts[2]}.{parts[3]}.{parts[4]}"  # Берем первые пять частей
    else:
        return None  # Возвращаем None, если меньше пяти уровней

    # Находим совпадение по section и возвращаем текст
    match = df.loc[df['section4'] == main_section, 'text']
    return match.values[0] if not match.empty else None

# Применяем функцию к DataFrame
dt['Глава'] = dt.apply(lambda row: get_main_text(row, dt), axis=1)
dt['Подглава'] = dt.apply(lambda row: get_main_text1(row, dt), axis=1)
dt['Подпункт'] = dt.apply(lambda row: get_main_text2(row, dt), axis=1)
dt['под-подподпункт'] = dt.apply(lambda row: get_main_text_5(row, dt), axis=1)

In [11]:
dt.head(10)

,punkt,text,section1,section2,section3,section4,Глава,Подглава,Подпункт,под-подподпункт
0,5.1.,General,5.1,-1,-1,-1,General,None,None,None
1,5.1.1.,Braking equipment,5.1,5.1.1,-1,-1,General,Braking equipment,None,None
2,5.1.1.1.,"The braking equipment shall be so designed, c...",5.1,5.1.1,5.1.1.1,-1,General,Braking equipment,"The braking equipment shall be so designed, c...",None
3,5.1.1.2.,"In particular, the braking equipment shall be...",5.1,5.1.1,5.1.1.2,-1,General,Braking equipment,"In particular, the braking equipment shall be...",None
4,5.1.1.3.,Brake linings shall not contain asbestos.,5.1,5.1.1,5.1.1.3,-1,General,Braking equipment,Brake linings shall not contain asbestos.,None
5,5.1.1.4.,The effectiveness of the braking equipment sh...,5.1,5.1.1,5.1.1.4,-1,General,Braking equipment,The effectiveness of the braking equipment sh...,None
6,5.1.1.5.,A failure detection signal may interrupt mome...,5.1,5.1.1,5.1.1.5,-1,General,Braking equipment,A failure detection signal may interrupt mome...,None
7,5.1.2.,Functions of the braking equipment\nThe braki...,5.1,5.1.2,-1,-1,General,Functions of the braking equipment\nThe braki...,None,None
8,5.1.2.1.,Service braking system\nThe service braking s...,5.1,5.1.2,5.1.2.1,-1,General,Functions of the braking equipment\nThe braki...,Service braking system\nThe service braking s...,None
9,5.1.2.2.,Secondary braking system\nThe secondary braki...,5.1,5.1.2,5.1.2.2,-1,General,Functions of the braking equipment\nThe braki...,Secondary braking system\nThe secondary braki...,None


In [12]:
cl = dt.drop(['section1','section2','section3','section4'],axis = 1)

In [17]:
cl.head(6)

,punkt,text,Глава,Подглава,Подпункт,под-подподпункт
0,5.1.,General,General,None,None,None
1,5.1.1.,Braking equipment,General,Braking equipment,None,None
2,5.1.1.1.,"The braking equipment shall be so designed, c...",General,Braking equipment,"The braking equipment shall be so designed, c...",None
3,5.1.1.2.,"In particular, the braking equipment shall be...",General,Braking equipment,"In particular, the braking equipment shall be...",None
4,5.1.1.3.,Brake linings shall not contain asbestos.,General,Braking equipment,Brake linings shall not contain asbestos.,None
5,5.1.1.4.,The effectiveness of the braking equipment sh...,General,Braking equipment,The effectiveness of the braking equipment sh...,None


In [13]:
import os 
os.chdir(r"C:\Users\lanmo\Downloads")

In [14]:
cl.to_excel('braking.xlsx')

In [15]:
dt['глава'] = dt.apply(lambda row: get_main_text(row, dt), axis=1)